# 0. 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 1. 기존 데이터셋 불러오는 코드

In [ ]:
import pandas as pd

path = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct/results/original_1.3/original_1_3_results.parquet"

VIEW_NUM = 15

df = pd.read_parquet(path)

for idx, row in df.iterrows():
  print(f"\n📌======= {idx + 1}번 행 =======📌")

  for i, col in enumerate(df.columns, start=1):
    value = row[col]

    print(f"✅{i}. {col}\n{value}\n")

# 2. 데이터 매핑

In [ ]:
import pandas as pd

path = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct/results/original_1.3/original_1_3_results.parquet"
VIEW_NUM = 10

df = pd.read_parquet(path)

# 1. 출력할 컬럼 이름들만 리스트로 정의
columns_to_print = [
    'original_model_output',
    'trained_model_output',
    'original_bhc',
    'original_di'
]

# 2. 원래 컬럼 번호를 가져오기 위해 딕셔너리 생성
col_indices = {col: df.columns.get_loc(col) + 1 for col in columns_to_print}

# 3. VIEW_NUM 만큼만 상위 N개 행을 가져옴
for idx, row in df.head(VIEW_NUM).iterrows():
    print(f"\n📌======= {idx + 1}번 행 =======📌")

    # 4. 정의한 리스트(columns_to_print)에 있는 컬럼들만 반복
    for col_name in columns_to_print:
        value = row[col_name]
        original_index = col_indices[col_name]  # 저장해둔 원래 번호 가져오기

        print(f"✅{original_index}. {col_name}\n{value}\n")

# 2.1 데이터 분할 확인

In [ ]:
import pandas as pd
import re

path = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct/results/original_1.3/original_1_3_results.parquet"
VIEW_NUM = 5 # 확인할 행 개수 조절

df = pd.read_parquet(path)

# 텍스트 분리 함수
def split_bhc_di(text):
    if not isinstance(text, str):
        return "", ""
    bhc_pattern = r"Brief Hospital Course\s*"
    di_pattern = r"Discharge Instructions\s*"
    bhc_match = re.search(bhc_pattern, text, re.IGNORECASE)
    di_match = re.search(di_pattern, text, re.IGNORECASE)
    bhc_text = ""
    di_text = ""

    if bhc_match and di_match:
        bhc_start = bhc_match.end()
        bhc_end = di_match.start()
        # BHC 텍스트 추출 후 맨 앞 콜론 및 관련 공백 제거
        bhc_text = text[bhc_start:bhc_end].strip().lstrip(':').strip()
        di_start = di_match.end()
        # DI 텍스트 추출 후 맨 앞 콜론 및 관련 공백 제거
        di_text = text[di_start:].strip().lstrip(':').strip()
    elif bhc_match:
        bhc_start = bhc_match.end()
        # BHC 텍스트 추출 후 맨 앞 콜론 및 관련 공백 제거
        bhc_text = text[bhc_start:].strip().lstrip(':').strip()
    elif di_match:
        di_start = di_match.end()
        # DI 텍스트 추출 후 맨 앞 콜론 및 관련 공백 제거
        di_text = text[di_start:].strip().lstrip(':').strip()
    return bhc_text, di_text

# 상위 VIEW_NUM 개 행에 대해 분할 결과 출력
for idx, row in df.head(VIEW_NUM).iterrows():
    print(f"\n\n\n📌======= {idx + 1}번 행 분할 결과 확인 =======📌")

    # 원본 모델 출력 가져오기 및 분리
    om_output = row['original_model_output']
    om_bhc, om_di = split_bhc_di(om_output)

    # 파인튜닝 모델 출력 가져오기 및 분리
    tm_output = row['trained_model_output']
    tm_bhc, tm_di = split_bhc_di(tm_output)

    # 정답 가져오기
    gt_bhc = str(row['original_bhc']) if pd.notna(row['original_bhc']) else ""
    gt_di = str(row['original_di']) if pd.notna(row['original_di']) else ""

    print("\n--- [원본 모델 BHC] ---")
    print(om_bhc)

    print("\n\n--- [원본 모델 DI] ---")
    print(om_di)

    print("\n\n--- [파인튜닝 모델 BHC] ---")
    print(tm_bhc)

    print("\n\n--- [파인튜닝 모델 DI] ---")
    print(tm_di)

    print("\n--- [정답 BHC] ---")
    print(gt_bhc)

    print("\n--- [정답 DI] ---")
    print(gt_di)
    print("=" * 40) # 행 구분

# 3. 평가 (BERTScore)

In [ ]:
# 평가에 필요한 라이브러리 다운
!pip uninstall transformers bert-score -y
!pip install transformers bert-score

In [23]:
import pandas as pd
from bert_score import score
import re
import warnings
import torch
import numpy as np # NaN 값을 사용하기 위해 numpy 임포트

# bert_score 라이브러리 경고 메시지 무시
warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

# --- 설정 ---
path = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct/results/original_1.3/original_1_3_results.parquet"
VIEW_NUM = None # None으로 설정하면 전체 행 처리, 숫자로 지정하면 상위 N개 행만 처리
MODEL_TYPE = "roberta-large" # BERTScore 계산에 사용할 모델
# --- 설정 끝 ---

# GPU 사용 가능 여부 확인 및 device 설정 (문자열로 변경)
if torch.cuda.is_available():
    device = "cuda:0" # GPU 사용 시 문자열 "cuda:0"
    print("Using device: GPU")
else:
    device = "cpu"    # CPU 사용 시 문자열 "cpu"
    print("Using device: CPU")


df = pd.read_parquet(path)

# 텍스트 분리 함수 (맨 앞 콜론 제거 포함, 검증 완료)
def split_bhc_di(text):
    if not isinstance(text, str):
        return "", ""
    bhc_pattern = r"Brief Hospital Course\s*"
    di_pattern = r"Discharge Instructions\s*"
    bhc_match = re.search(bhc_pattern, text, re.IGNORECASE)
    di_match = re.search(di_pattern, text, re.IGNORECASE)
    bhc_text = ""
    di_text = ""
    if bhc_match and di_match:
        bhc_start = bhc_match.end()
        bhc_end = di_match.start()
        bhc_text = text[bhc_start:bhc_end].strip().lstrip(':').strip()
        di_start = di_match.end()
        di_text = text[di_start:].strip().lstrip(':').strip()
    elif bhc_match:
        bhc_start = bhc_match.end()
        bhc_text = text[bhc_start:].strip().lstrip(':').strip()
    elif di_match:
        di_start = di_match.end()
        di_text = text[di_start:].strip().lstrip(':').strip()
    return bhc_text, di_text

# 결과를 저장할 리스트
bert_scores = []

# 처리할 행 결정
if VIEW_NUM is None:
    rows_to_process = df.iterrows()
    total_rows = len(df)
else:
    rows_to_process = df.head(VIEW_NUM).iterrows()
    total_rows = VIEW_NUM

print(f"총 {total_rows}개 행에 대한 BERTScore 계산을 시작합니다 (모델: {MODEL_TYPE}, 장치: {device})...")

# 행 반복, 텍스트 분리, BERTScore 계산
for idx, row in rows_to_process:
    if (idx + 1) % 50 == 0: # 50행마다 진행 상황 출력
        print(f"행 {idx + 1}/{total_rows} 처리 중...")

    # 모델 출력값 가져오기
    om_output = row['original_model_output']
    tm_output = row['trained_model_output']

    # 정답(Ground Truth) 가져오기 (NaN 값 처리)
    gt_bhc = str(row['original_bhc']) if pd.notna(row['original_bhc']) else ""
    gt_di = str(row['original_di']) if pd.notna(row['original_di']) else ""

    # 모델 출력값 분리
    om_bhc, om_di = split_bhc_di(om_output)
    tm_bhc, tm_di = split_bhc_di(tm_output)

    # --- BERTScore 계산 (6가지 비교) ---
    # 점수 변수를 NaN으로 초기화
    f1_om_gt_bhc, f1_tm_gt_bhc, f1_om_gt_di, f1_tm_gt_di = np.nan, np.nan, np.nan, np.nan
    f1_om_tm_bhc, f1_om_tm_di = np.nan, np.nan

    # 계산 함수 정의 (오류 처리 포함)
    def calculate_f1(candidates, references):
        # 함수 내부에서 빈 문자열 체크 제거 (호출 전에 체크)
        try:
            _, _, F1 = score(candidates, references, model_type=MODEL_TYPE, lang='en', verbose=False, device=device)
            return F1[0].item() if F1.numel() > 0 else 0.0 # 0.0 반환 유지 (오류 시)
        except Exception as e:
            print(f" - 행 {idx + 1} 계산 오류: {e}")
            return np.nan # 오류 발생 시 NaN 반환

    # 점수 계산
    if om_bhc and gt_bhc:
        f1_om_gt_bhc = calculate_f1([om_bhc], [gt_bhc])
    if tm_bhc and gt_bhc:
        f1_tm_gt_bhc = calculate_f1([tm_bhc], [gt_bhc])
    if om_di and gt_di:
        f1_om_gt_di  = calculate_f1([om_di], [gt_di])
    if tm_di and gt_di:
        f1_tm_gt_di  = calculate_f1([tm_di], [gt_di])
    if om_bhc and tm_bhc: # OM vs TM 비교도 양쪽 다 있어야 계산
        f1_om_tm_bhc = calculate_f1([om_bhc], [tm_bhc])
    if om_di and tm_di: # OM vs TM 비교도 양쪽 다 있어야 계산
        f1_om_tm_di  = calculate_f1([om_di], [tm_di])


    # 결과 리스트에 추가
    bert_scores.append({
        'row_index': idx + 1,
        'F1_OM_vs_GT_BHC': f1_om_gt_bhc,
        'F1_TM_vs_GT_BHC': f1_tm_gt_bhc,
        'F1_OM_vs_GT_DI': f1_om_gt_di,
        'F1_TM_vs_GT_DI': f1_tm_gt_di,
        'F1_OM_vs_TM_BHC': f1_om_tm_bhc,
        'F1_OM_vs_TM_DI': f1_om_tm_di,
    })

print(f"총 {total_rows}개 행에 대한 BERTScore 계산 완료.")

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(bert_scores)

# 결과 출력 (상위 5개 행)
print("\n===== BERTScore F1 결과 (상위 5개 행) =====")
print(results_df.head()) # NaN 값이 포함된 채로 출력될 수 있음

# 평균 점수 출력
print("\n===== 평균 BERTScore F1 (NaN 제외) =====")
avg_scores = results_df.mean(numeric_only=True) # skipna=True가 기본값
print(f"원본 모델 BHC vs 정답 BHC: {avg_scores.get('F1_OM_vs_GT_BHC', 0.0):.4f}")
print(f"파인튜닝 모델 BHC vs 정답 BHC: {avg_scores.get('F1_TM_vs_GT_BHC', 0.0):.4f}")
print(f"원본 모델 DI vs 정답 DI: {avg_scores.get('F1_OM_vs_GT_DI', 0.0):.4f}")
print(f"파인튜닝 모델 DI vs 정답 DI: {avg_scores.get('F1_TM_vs_GT_DI', 0.0):.4f}")
print(f"원본 모델 BHC vs 파인튜닝 모델 BHC: {avg_scores.get('F1_OM_vs_TM_BHC', 0.0):.4f}")
print(f"원본 모델 DI vs 파인튜닝 모델 DI: {avg_scores.get('F1_OM_vs_TM_DI', 0.0):.4f}")

print("\n===== 각 비교별 NaN (계산 건너뜀) 개수 =====")
print(results_df.isnull().sum())

Using device: GPU
총 10개 행에 대한 BERTScore 계산을 시작합니다 (모델: roberta-large, 장치: cuda:0)...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

총 10개 행에 대한 BERTScore 계산 완료.

===== BERTScore F1 결과 (상위 5개 행) =====
   row_index  F1_OM_vs_GT_BHC  F1_TM_vs_GT_BHC  F1_OM_vs_GT_DI  \
0          1         0.820352         0.854543        0.794991   
1          2         0.788898         0.823793        0.788979   
2          3         0.817770         0.829365        0.809633   
3          4         0.798389         0.802302        0.791982   
4          5         0.811931         0.850316        0.794196   

   F1_TM_vs_GT_DI  F1_OM_vs_TM_BHC  F1_OM_vs_TM_DI  
0        0.832612         0.805765        0.792189  
1        0.834645         0.796798        0.784696  
2        0.887345         0.822171        0.829458  
3        0.844316         0.784360        0.809186  
4             NaN         0.805998             NaN  

===== 평균 BERTScore F1 (NaN 제외) =====
원본 모델 BHC vs 정답 BHC: 0.8086
파인튜닝 모델 BHC vs 정답 BHC: 0.8415
원본 모델 DI vs 정답 DI: 0.8003
파인튜닝 모델 DI vs 정답 DI: 0.8564
원본 모델 BHC vs 파인튜닝 모델 BHC: 0.8082
원본 모델 DI vs 파인튜닝 모델 DI: 0.8018

==

# 3.1 시각화

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

if 'results_df' not in locals():
    print("오류: 'results_df' 데이터프레임이 존재하지 않습니다.")
else:
    print("시각화 진행 중...")
    # --- 시각화 ---

    # 그래프 스타일 설정
    sns.set_style("whitegrid")

    # 1행 2열의 그래프 영역 생성 (BHC용, DI용)
    fig, axes = plt.subplots(1, 2, figsize=(14, 7)) # 가로 크기 늘림

    # 1. BHC 점수 비교 상자 그림 (OM vs GT, TM vs GT)
    # BHC 비교에 사용할 컬럼만 선택
    bhc_scores_to_plot = results_df[['F1_OM_vs_GT_BHC', 'F1_TM_vs_GT_BHC']]
    # 그래프 레이블을 위한 컬럼 이름 변경
    bhc_scores_to_plot.columns = ['Original vs. GT', 'Fine-tuned vs. GT']

    sns.boxplot(data=bhc_scores_to_plot, ax=axes[0], palette="pastel") # 색상 팔레트 변경
    axes[0].set_title('BHC: BERTScore F1 Comparison (vs. Ground Truth)') # 제목 수정
    axes[0].set_ylabel('BERTScore F1')
    axes[0].set_ylim(0, 1) # F1 점수 범위 (0 ~ 1)

    # 2. DI 점수 비교 상자 그림 (OM vs GT, TM vs GT)
    # DI 비교에 사용할 컬럼만 선택
    di_scores_to_plot = results_df[['F1_OM_vs_GT_DI', 'F1_TM_vs_GT_DI']]
    # 컬럼 이름 변경
    di_scores_to_plot.columns = ['Original vs. GT', 'Fine-tuned vs. GT']

    sns.boxplot(data=di_scores_to_plot, ax=axes[1], palette="pastel")
    axes[1].set_title('DI: BERTScore F1 Comparison (vs. Ground Truth)') # 제목 수정
    axes[1].set_ylabel('BERTScore F1')
    axes[1].set_ylim(0, 1)

    # 그래프 레이아웃 조정 및 파일 저장
    plt.tight_layout()
    plt.savefig("bertscore_vs_gt_comparison_boxplot.png") # 파일 이름 변경

    # 저장 완료 메시지 출력
    print("상자 그림이 bertscore_vs_gt_comparison_boxplot.png 로 저장되었습니다.")

    # plt.show()

# 4. 평가 (BLEU)

In [ ]:
!pip install nltk

In [22]:
# 필요 라이브러리 설치 (Colab에서 처음 실행 시 필요)
# !pip install nltk

import pandas as pd
import re
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np # NaN 값을 사용하기 위해 numpy 임포트

# nltk 데이터 다운로드 (punkt 토크나이저)
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

# --- 설정 ---
path = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct/results/original_1.3/original_1_3_results.parquet"
VIEW_NUM = None # None으로 설정하면 전체 행 처리, 숫자로 지정하면 상위 N개 행만 처리

df = pd.read_parquet(path)

# 텍스트 분리 함수
def split_bhc_di(text):
    if not isinstance(text, str):
        return "", ""
    bhc_pattern = r"Brief Hospital Course\s*"
    di_pattern = r"Discharge Instructions\s*"
    bhc_match = re.search(bhc_pattern, text, re.IGNORECASE)
    di_match = re.search(di_pattern, text, re.IGNORECASE)
    bhc_text = ""
    di_text = ""
    if bhc_match and di_match:
        bhc_start = bhc_match.end()
        bhc_end = di_match.start()
        bhc_text = text[bhc_start:bhc_end].strip().lstrip(':').strip()
        di_start = di_match.end()
        di_text = text[di_start:].strip().lstrip(':').strip()
    elif bhc_match:
        bhc_start = bhc_match.end()
        bhc_text = text[bhc_start:].strip().lstrip(':').strip()
    elif di_match:
        di_start = di_match.end()
        di_text = text[di_start:].strip().lstrip(':').strip()
    return bhc_text, di_text

# 결과를 저장할 리스트
bleu_scores_list = []

# 처리할 행 결정
if VIEW_NUM is None:
    rows_to_process = df.iterrows()
    total_rows = len(df)
else:
    rows_to_process = df.head(VIEW_NUM).iterrows()
    total_rows = VIEW_NUM

print(f"총 {total_rows}개 행에 대한 BLEU-4 점수 계산을 시작합니다...")

# Smoothing function 정의
chencherry = SmoothingFunction()

# 토크나이징 및 점수 계산 함수 정의
def calculate_bleu(hypothesis_str, reference_str):
    # 함수 내부 빈 문자열 체크 제거 (호출 전에 체크)
    hypothesis_tokens = hypothesis_str.split()
    reference_tokens_list = [reference_str.split()]
    try:
        score_val = sentence_bleu(reference_tokens_list, hypothesis_tokens, smoothing_function=chencherry.method1)
        return score_val
    except Exception as e:
        print(f" - 행 계산 중 오류 발생 (BLEU): {e}") # 오류 발생 시 행 번호 명시적으로 출력하지 않음
        return np.nan # 오류 시 NaN 반환

# 행 반복, 텍스트 분리, BLEU 점수 계산
for idx, row in rows_to_process:
    if (idx + 1) % 50 == 0: # 50행마다 진행 상황 출력
        print(f"행 {idx + 1}/{total_rows} 처리 중...")

    # 모델 출력값 가져오기
    om_output = row['original_model_output']
    tm_output = row['trained_model_output']

    # 정답(Ground Truth) 가져오기 (NaN 값 처리)
    gt_bhc = str(row['original_bhc']) if pd.notna(row['original_bhc']) else ""
    gt_di = str(row['original_di']) if pd.notna(row['original_di']) else ""

    # 모델 출력값 분리
    om_bhc, om_di = split_bhc_di(om_output)
    tm_bhc, tm_di = split_bhc_di(tm_output)

    # --- BLEU 점수 계산 ---
    bleu_om_gt_bhc, bleu_tm_gt_bhc, bleu_om_gt_di, bleu_tm_gt_di = np.nan, np.nan, np.nan, np.nan

    # 점수 계산
    if om_bhc and gt_bhc:
        bleu_om_gt_bhc = calculate_bleu(om_bhc, gt_bhc)
    if tm_bhc and gt_bhc:
        bleu_tm_gt_bhc = calculate_bleu(tm_bhc, gt_bhc)
    if om_di and gt_di:
        bleu_om_gt_di  = calculate_bleu(om_di, gt_di)
    if tm_di and gt_di:
        bleu_tm_gt_di  = calculate_bleu(tm_di, gt_di)

    # 결과 리스트에 추가
    bleu_scores_list.append({
        'row_index': idx + 1,
        'BLEU4_OM_vs_GT_BHC': bleu_om_gt_bhc,
        'BLEU4_TM_vs_GT_BHC': bleu_tm_gt_bhc,
        'BLEU4_OM_vs_GT_DI': bleu_om_gt_di,
        'BLEU4_TM_vs_GT_DI': bleu_tm_gt_di,
    })

print(f"총 {total_rows}개 행에 대한 BLEU-4 점수 계산 완료.")

# 결과를 데이터프레임으로 변환
bleu_results_df = pd.DataFrame(bleu_scores_list)

# 결과 출력 (상위 5개 행)
print("\n===== BLEU-4 점수 결과 (상위 5개 행) =====")
print(bleu_results_df.head()) # NaN 값이 포함된 채로 출력될 수 있음

# 평균 점수 출력
print("\n===== 평균 BLEU-4 점수 (NaN 제외) =====")
avg_bleu_scores = bleu_results_df.mean(numeric_only=True) # skipna=True가 기본값
print(f"원본 모델 BHC vs 정답 BHC: {avg_bleu_scores.get('BLEU4_OM_vs_GT_BHC', 0.0):.4f}")
print(f"파인튜닝 모델 BHC vs 정답 BHC: {avg_bleu_scores.get('BLEU4_TM_vs_GT_BHC', 0.0):.4f}")
print(f"원본 모델 DI vs 정답 DI: {avg_bleu_scores.get('BLEU4_OM_vs_GT_DI', 0.0):.4f}")
print(f"파인튜닝 모델 DI vs 정답 DI: {avg_bleu_scores.get('BLEU4_TM_vs_GT_DI', 0.0):.4f}")

print("\n===== 각 비교별 NaN (계산 건너뜀) 개수 =====")
print(bleu_results_df.isnull().sum())


총 10개 행에 대한 BLEU-4 점수 계산을 시작합니다...
총 10개 행에 대한 BLEU-4 점수 계산 완료.

===== BLEU-4 점수 결과 (상위 5개 행) =====
   row_index  BLEU4_OM_vs_GT_BHC  BLEU4_TM_vs_GT_BHC  BLEU4_OM_vs_GT_DI  \
0          1            0.009228            0.177181           0.004766   
1          2            0.003957            0.008319           0.002432   
2          3            0.004316            0.174411           0.003733   
3          4            0.001810            0.063336           0.005457   
4          5            0.000999            0.001277           0.001544   

   BLEU4_TM_vs_GT_DI  
0           0.001604  
1           0.013378  
2           0.141416  
3           0.005420  
4                NaN  

===== 평균 BLEU-4 점수 (NaN 제외) =====
원본 모델 BHC vs 정답 BHC: 0.0042
파인튜닝 모델 BHC vs 정답 BHC: 0.0795
원본 모델 DI vs 정답 DI: 0.0066
파인튜닝 모델 DI vs 정답 DI: 0.0413

===== 각 비교별 NaN (계산 건너뜀) 개수 =====
row_index             0
BLEU4_OM_vs_GT_BHC    1
BLEU4_TM_vs_GT_BHC    1
BLEU4_OM_vs_GT_DI     1
BLEU4_TM_vs_GT_DI     1
dtype: in

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

if 'bleu_results_df' not in locals():
    print("오류: 'bleu_results_df' 데이터프레임이 존재하지 않습니다.")
else:
    print("BLEU-4 점수 시각화 진행 중...")
    # --- 시각화 ---

    # 그래프 스타일 설정
    sns.set_style("whitegrid")

    # 1행 2열의 그래프 영역 생성 (BHC용, DI용)
    fig, axes = plt.subplots(1, 2, figsize=(14, 7))

    # 1. BHC 점수 비교 상자 그림 (OM vs GT, TM vs GT)
    # BHC 비교에 사용할 컬럼만 선택
    bhc_bleu_scores_to_plot = bleu_results_df[['BLEU4_OM_vs_GT_BHC', 'BLEU4_TM_vs_GT_BHC']]
    # 그래프 레이블을 위한 컬럼 이름 변경
    bhc_bleu_scores_to_plot.columns = ['Original vs. GT', 'Fine-tuned vs. GT']

    sns.boxplot(data=bhc_bleu_scores_to_plot, ax=axes[0], palette="pastel")
    axes[0].set_title('BHC: BLEU-4 Score Comparison (vs. Ground Truth)')
    axes[0].set_ylabel('BLEU-4 Score')
    # BLEU 점수는 보통 낮으므로, 상한선을 1.0 대신 데이터 최대값 기준으로 자동 조절하거나
    # 데이터 분포를 보고 적절히 설정 (예: 0.2 또는 0.3)
    axes[0].set_ylim(0, 1)

    # 2. DI 점수 비교 상자 그림 (OM vs GT, TM vs GT)
    # DI 비교에 사용할 컬럼만 선택
    di_bleu_scores_to_plot = bleu_results_df[['BLEU4_OM_vs_GT_DI', 'BLEU4_TM_vs_GT_DI']]
    # 컬럼 이름 변경
    di_bleu_scores_to_plot.columns = ['Original vs. GT', 'Fine-tuned vs. GT']

    sns.boxplot(data=di_bleu_scores_to_plot, ax=axes[1], palette="pastel")
    axes[1].set_title('DI: BLEU-4 Score Comparison (vs. Ground Truth)')
    axes[1].set_ylabel('BLEU-4 Score')
    axes[1].set_ylim(0, 1) # y축 범위 설정

    # 그래프 레이아웃 조정 및 파일 저장
    plt.tight_layout()
    plt.savefig("bleu4_comparison_boxplot.png") # 파일 이름 지정

    # 저장 완료 메시지 출력
    print("상자 그림이 bleu4_comparison_boxplot.png 로 저장되었습니다.")

    plt.show()